In [1]:
import pandas as pd
import pandas_market_calendars as mcal
import datetime
from Trade_Functions import *
import numpy as np

In [2]:
cal = get_calendar()

In [37]:
def pick_trade(RunDate,NumStocks,Money,cal=get_calendar(),holdTime=0,TimeInput=5,NDXfile="Nasdaq100MembersJan2024.csv",debug=False):
    
    print(RunDate)
    
    LongStockList = list()
    ShortStockList = list()
    
    PctChange = get_SortedPctChange(RunDate,TimeInput,NDXfile,debug=debug)
    
    LongStockList = PctChange["Ticker"].iloc[:NumStocks]
    LongReturns = getLongReturns(RunDate,LongStockList,holdTime,debug=debug)
    TotalReturn= (1+LongReturns[0])*(Money*0.75) + (1+LongReturns[1])*(Money*0.25)
        
     
    
    #ShortStockList =  PctChange["Ticker"].iloc[len(PctChange["Ticker"])-NumStocks:]
    #ShortReturns = getShortReturns(RunDate,ShortStockList,holdTime,debug=debug)
    #TotalReturn += sum(ShortReturns)/NumStocks

    print("Total Return: ", TotalReturn)
    
    return TotalReturn

In [38]:
TimeInput = 2
NDXfile="Nasdaq100MembersJan2024.csv"
NumStocks=2
holdTime=1

In [42]:
Money = 1
TotalReturnList = list()
DateReturnList = list()

DateRange=['2023-11-30','2024-11-06']
DateRange=[cal.index(DateRange[0]),cal.index(DateRange[1])]
for day in cal[DateRange[0]:DateRange[1]]:
    Money = pick_trade(RunDate=day,NumStocks=NumStocks,Money=Money,cal=cal,TimeInput=TimeInput,NDXfile=NDXfile,holdTime=holdTime)
    TotalReturnList.append(Money)
    DateReturnList.append(day)

dfOut = pd.DataFrame({'Date':DateReturnList,'Return':TotalReturnList})







2023-11-30
Total Return:  1.0287305012747359
2023-12-01
Total Return:  1.0261021718618917
2023-12-04
Total Return:  1.0054196286230805
2023-12-05
Total Return:  1.0222052889752524
2023-12-06
Total Return:  0.9839382717587349
2023-12-07
Total Return:  0.9817552450191256
2023-12-08
Total Return:  1.0076999382607603
2023-12-11
Total Return:  0.9812780695276815
2023-12-12
Total Return:  1.0358103179307594
2023-12-13
Total Return:  1.0523766717800547
2023-12-14
Total Return:  1.059755930755806
2023-12-15
Total Return:  1.038926784455363
2023-12-18
Total Return:  1.0594844688290097
2023-12-19
Total Return:  1.0213990899494543
2023-12-20
Total Return:  1.0345809617264214
2023-12-21
Total Return:  1.0408758651793777
2023-12-22
Total Return:  1.0536349463434547
2023-12-26
Total Return:  1.0668403982673877
2023-12-27
Total Return:  1.058864838312112
2023-12-28
Total Return:  1.0432767003802357
2023-12-29
Total Return:  0.9299631736875511
2024-01-02
Total Return:  0.9286933045840917
2024-01-03
To

Total Return:  1.9835556372350922
2024-08-26
Total Return:  2.010945760558854
2024-08-27
Total Return:  1.9770832590096874
2024-08-28
Total Return:  1.9259514324251519
2024-08-29
Total Return:  1.9318396424071826
2024-08-30
Total Return:  1.8333754730642362
2024-09-03
Total Return:  1.7880989699743846
2024-09-04
Total Return:  1.7465704269563567
2024-09-05
Total Return:  1.6720130355751817
2024-09-06
Total Return:  1.62705344294397
2024-09-09
Total Return:  1.6905721736598949
2024-09-10
Total Return:  1.8478868555566959
2024-09-11
Total Return:  2.0097309712094016
2024-09-12
Total Return:  2.141629737917869
2024-09-13
Total Return:  2.299340031462961
2024-09-16
Total Return:  2.264433707431685
2024-09-17
Total Return:  2.1844466059667598
2024-09-18
Total Return:  2.1185167880619757
2024-09-19
Total Return:  2.01057269725474
2024-09-20
Total Return:  2.065858129575564
2024-09-23
Total Return:  2.0135987958520825
2024-09-24
Total Return:  2.0536404412406837
2024-09-25
Total Return:  2.31

In [45]:
PrctReturnList=[]
retList= [1]+TotalReturnList
for i in range(1,len(retList)-1):
    PrctReturnList.append((TotalReturnList[i]-TotalReturnList[i-1])/TotalReturnList[i-1])

In [46]:
PrctReturnList

[-0.0025549251330521778,
 -0.020156416978712823,
 0.016695178683909125,
 -0.0374357456659999,
 -0.002218662290376521,
 0.02642684454528104,
 -0.02621997653257939,
 0.055572676182731685,
 0.015993617327919575,
 0.00701199406413065,
 -0.019654663584272623,
 0.01978742360023355,
 -0.03594708558743586,
 0.012905701509504475,
 0.006084495738691965,
 0.01225802383445423,
 0.012533232662566299,
 -0.007475869837914417,
 -0.014721555922779189,
 -0.10861310968737824,
 -0.001365504720390183,
 -0.033110222719496936,
 -0.017447603088565262,
 0.10549249756212348,
 0.0460654030682297,
 0.04969169234564217,
 0.0008013780120010372,
 0.0021507111502469013,
 -0.0136103834211735,
 -0.01684566420351382,
 0.02612677199008747,
 0.08893250550757831,
 0.026953787715332352,
 -0.006865615098582963,
 -0.008295229076155777,
 0.007268873117429107,
 0.024737732899355117,
 0.009613612524859915,
 -0.011381042160716361,
 -0.04125436825030156,
 0.115065395012015,
 0.05892943665554195,
 0.08969456586295697,
 -0.025739980

In [47]:
dfOut

,Date,Return
0,2023-11-30,1.028731
1,2023-12-01,1.026102
2,2023-12-04,1.005420
3,2023-12-05,1.022205
4,2023-12-06,0.983938
...,...,...
230,2024-10-30,2.260939
231,2024-10-31,2.243564
232,2024-11-01,2.207333
233,2024-11-04,2.252880


In [28]:
print(np.mean(dfOut["Return"][dfOut["Return"] <0]))
len(dfOut["Return"][dfOut["Return"] <0])

-0.023325753315407052


123

In [29]:
print(np.mean(dfOut["Return"][dfOut["Return"] >0]))
len(dfOut["Return"][dfOut["Return"] >0])

0.02400659183104909


150

In [ ]:
dfOut.to_csv("ReturnData.csv")
